In [1]:
import pandas as pd
import yaml
import sys
import os
from glob import glob
from pathlib import Path
import pickle

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle/src')
from utils.xgb import fit_xgb
from utils.metric import compute_comptetition_metric
from utils.postprocess import post_process
from utils.set_seed import seed_base
from feature_engineering.stage2 import generate_2nd_stage_features

PACKAGE_DIR = Path("/kaggle/src")
CFG = yaml.safe_load(open(PACKAGE_DIR / "config.yaml", "r"))
print(CFG["2nd_stage"]["execution"]["exp_id"])

CFG["output_dir"] = f"/kaggle/output/{CFG['2nd_stage']['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

seed_base(CFG["env"]["seed"])

2023-09-30 06:17:41.562802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-30 06:17:41.991127: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-30 06:17:43.348950: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/u

exp_044


In [2]:
# 特徴生成
files = glob(f"{CFG['dataset']['step_csv_dir']}/*.csv")
train, features = generate_2nd_stage_features(files)

# cv splitとマージ
cv_split = pd.read_csv(CFG['dataset']['cv_split_path'])
train["fold"] = train["series_id"].map(cv_split.set_index("series_id")["fold"])
display(train.head(5))
train.shape

generate features: 100%|██████████| 277/277 [08:02<00:00,  1.74s/it]


,anglez_diff_abs,anglez_diff_abs_rolling_max_10,anglez_diff_abs_rolling_max_100,anglez_diff_abs_rolling_max_1000,anglez_diff_abs_rolling_max_50,anglez_diff_abs_rolling_mean_10,anglez_diff_abs_rolling_mean_100,anglez_diff_abs_rolling_mean_1000,anglez_diff_abs_rolling_mean_50,anglez_diff_abs_rolling_median_10,...,anglez_mean_rolling_median_1000_shift_416,enmo_mean_rolling_median_1000_shift_416,anglez_rolling_square_mean_1000_shift_416,enmo_rolling_square_mean_1000_shift_416,anglez_diff_abs_rolling_square_mean_1000_shift_416,enmo_diff_abs_rolling_square_mean_1000_shift_416,anglez_mean_rolling_square_mean_1000_shift_416,enmo_mean_rolling_square_mean_1000_shift_416,reduce_step,fold
283,16.702658,38.064583,51.115900,84.6226,51.115900,15.962638,11.263916,8.043460,12.792510,14.257165,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6803,3
284,7.195825,28.564683,51.115900,84.6226,51.115900,8.541195,12.527601,7.737612,12.364961,6.165692,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6827,3
285,16.351179,34.583987,51.115900,84.6226,40.295400,15.494380,12.499973,7.288104,12.204054,14.248077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6851,3
286,10.855896,32.284983,42.890471,84.6226,38.495083,10.967216,12.269771,6.938846,12.743385,6.737581,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6875,3
287,11.523954,27.963821,43.971200,84.6226,41.090162,12.171813,13.303702,6.618652,12.236429,10.896129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6899,3


(877617, 1380)

In [3]:
# 学習
trn_oof, models = fit_xgb(
    X=train, 
    y=train["target"], 
    folds=train["fold"].astype(int), 
    features=features.all_features(),
    params=CFG["2nd_stage"]["xgboost"], 
    es_rounds=100,
)
train["oof"] = trn_oof
train = train.drop(columns=list(set(features.all_features()) - set(["minutes"])))
train = train.sort_values(["series_id", "step"]).reset_index(drop=True)

# nanは1st stageの結果を使う
cand_path = os.path.join("/kaggle/output", CFG["1st_stage"]["execution"]["best_exp_id"], "oof.csv")
last_train = pd.read_csv(cand_path).rename(columns={"oof": "oof_1st"})
train = last_train[["series_id", "oof_1st", "step"]].merge(
    train[["series_id", "oof", "step", "minutes"]], 
    on=["series_id", "step"], 
    how="left")
train["oof"] = train["oof"].fillna(train["oof_1st"])
train = train.sort_values(["series_id", "step"]).reset_index(drop=True)

# 保存
for i, model in enumerate(models):
    model.save_model(os.path.join(CFG["output_dir"], f'xgb_fold{i}.model'))
with open(os.path.join(CFG["output_dir"], "features.pkl"), "wb") as f:
    pickle.dump(features, f)
train.to_csv(os.path.join(CFG["output_dir"], "oof.csv"), index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

== fold 0 ==
[0]	eval-logloss:0.64149
[100]	eval-logloss:0.26274
[200]	eval-logloss:0.26046
[258]	eval-logloss:0.26073


 20%|██        | 1/5 [00:32<02:11, 32.82s/it]

== fold 1 ==
[0]	eval-logloss:0.64230
[100]	eval-logloss:0.24628
[200]	eval-logloss:0.24169
[300]	eval-logloss:0.24443
[317]	eval-logloss:0.24385


 40%|████      | 2/5 [01:06<01:40, 33.41s/it]

...

 60%|██████    | 3/5 [01:42<01:09, 34.64s/it]

== fold 3 ==
[0]	eval-logloss:0.64678
[100]	eval-logloss:0.25801
[200]	eval-logloss:0.25550
[300]	eval-logloss:0.25771
[307]	eval-logloss:0.25772


 80%|████████  | 4/5 [02:18<00:34, 35.00s/it]

== fold 4 ==
[0]	eval-logloss:0.64456
[100]	eval-logloss:0.30335
[200]	eval-logloss:0.30245
[256]	eval-logloss:0.30406


100%|██████████| 5/5 [02:50<00:00, 34.01s/it]


In [4]:
# 後処理
sub = post_process(train)
sub = sub.merge(train[["series_id", "step", "minutes"]].rename(columns={"step": "key_step"}), on=["series_id", "key_step"], how="left")
sub.to_csv(os.path.join(CFG["output_dir"], "submission.csv"), index=False)

# スコア計算
labels = pd.read_csv(f"{CFG['dataset']['competition_dir']}/train_events.csv").dropna()
score, ap_table = compute_comptetition_metric(labels, sub)
print(f"score: {score:.4f}")
display(ap_table)

post process: 100%|██████████| 277/277 [00:46<00:00,  5.98it/s]


score: 0.6445


event   tolerance
onset   12           0.059275
        36           0.328763
        60           0.515234
        90           0.636371
        120          0.717173
        150          0.757794
        180          0.788101
        240          0.823117
        300          0.845200
        360          0.854514
wakeup  12           0.091472
        36           0.382547
        60           0.540376
        90           0.664446
        120          0.726033
        150          0.767801
        180          0.797469
        240          0.843693
        300          0.867589
        360          0.882284
dtype: float64